<a href="https://colab.research.google.com/github/HWMV/OrchestrAI/blob/main/frontend_doc/front_end_sample2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## main.dart : 진입점, 전체 테마

In [ ]:
import 'package:flutter/material.dart';
import 'package:provider/provider.dart';
import 'models/crew_model.dart';
import 'screens/crew_screen.dart';

void main() {
  runApp(
    ChangeNotifierProvider(
      create: (context) => CrewModel()..addDummyData(),
      child: OrchestrAIApp(),
    ),
  );
}

class OrchestrAIApp extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'OrchestrAI',
      theme: ThemeData(
        primarySwatch: Colors.blue,
        visualDensity: VisualDensity.adaptivePlatformDensity,
      ),
      home: CrewScreen(),
    );
  }
}

## screens/crew_screen.dart :
메인 화면

## 왼쪽 사이드 바 (컴포넌트 목록, 서랍형태,, 펼쳐보기 기능으로 사이드 바 추가)

## 화면 추가 main screen > agent 생성 화면 (task, tool) : 우측 사이드바 (파라미터 입력)

In [ ]:
import 'package:flutter/material.dart';
import 'package:provider/provider.dart';
import '../models/crew_model.dart';
import 'agent_screen.dart';

class CrewScreen extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('OrchestrAI')),
      body: Consumer<CrewModel>(
        builder: (context, crewModel, child) {
          return Column(
            children: [
              TextField(
                decoration: InputDecoration(labelText: 'Crew Name'),
                controller: TextEditingController(text: crewModel.crewName),
                onChanged: (value) => crewModel.setCrewName(value),
              ),
              Expanded(
                child: GridView.builder(
                  gridDelegate: SliverGridDelegateWithFixedCrossAxisCount(
                    crossAxisCount: 2,
                    childAspectRatio: 0.75,
                  ),
                  itemCount: 4,
                  itemBuilder: (context, index) {
                    if (index < crewModel.agents.length) {
                      final agent = crewModel.agents[index];
                      return GestureDetector(
                        onTap: () {
                          Navigator.push(
                            context,
                            MaterialPageRoute(
                              builder: (context) =>
                                  AgentScreen(agentIndex: index),
                            ),
                          );
                        },
                        child: Card(
                          child: Column(
                            mainAxisAlignment: MainAxisAlignment.center,
                            children: [
                              Image.asset(
                                'assets/agent_${agent.name.toLowerCase()}.png',
                                height: 100,
                              ),
                              Text(agent.name),
                              Text(agent.role),
                            ],
                          ),
                        ),
                      );
                    } else {
                      return GestureDetector(
                        onTap: () {
                          crewModel.addAgent(AgentModel());
                          Navigator.push(
                            context,
                            MaterialPageRoute(
                              builder: (context) => AgentScreen(
                                  agentIndex: crewModel.agents.length - 1),
                            ),
                          );
                        },
                        child: Card(
                          child: Column(
                            mainAxisAlignment: MainAxisAlignment.center,
                            children: [
                              Image.asset(
                                'assets/empty_agent.png',
                                height: 100,
                              ),
                              Text('Add New Agent'),
                            ],
                          ),
                        ),
                      );
                    }
                  },
                ),
              ),
            ],
          );
        },
      ),
    );
  }
}


## screens/agent_screen.dart

: 각 의자마다 Agent01~04까지 개별 화면으로 이동하도록 구현함

: Agent 조립 화면

In [ ]:
import 'package:flutter/material.dart';
import 'package:provider/provider.dart';
import '../models/crew_model.dart';
import '../widgets/lego_agent_builder.dart';
import '../widgets/parameter_sidebar.dart';

class AgentScreen extends StatelessWidget {
  final int agentIndex;

  AgentScreen({required this.agentIndex});

  @override
  Widget build(BuildContext context) {
    return Consumer<CrewModel>(
      builder: (context, crewModel, child) {
        final agent = crewModel.agents[agentIndex];
        return Scaffold(
          appBar: AppBar(title: Text('Agent ${agentIndex + 1}: ${agent.name}')),
          body: Row(
            children: [
              Expanded(
                flex: 2,
                child: LegoAgentBuilder(agentIndex: agentIndex),
              ),
              Expanded(
                flex: 1,
                child: ParameterSidebar(agentIndex: agentIndex),
              ),
            ],
          ),
        );
      },
    );
  }
}


## widgets/component_sidebar.dart
(삭제)

## : 좌측 컴포넌트 사이드 바 (펼쳐보기 기능 추가), 2개의 screen에 추가

## 수정 : main screen에서 의자 위젯(버트) Agent add 기능 추가

## widgents/crew_table.dart

In [ ]:
import 'package:flutter/material.dart';

class CrewTable extends StatelessWidget {
  final Function(int) onAgentTap;

  CrewTable({required this.onAgentTap});

  @override
  Widget build(BuildContext context) {
    return GridView.builder(
      gridDelegate: SliverGridDelegateWithFixedCrossAxisCount(
        crossAxisCount: 2,
        childAspectRatio: 1.5,
      ),
      itemCount: 4,
      itemBuilder: (context, index) {
        return GestureDetector(
          onTap: () => onAgentTap(index),
          child: Card(
            child: Center(
              child: Text('Agent ${index + 1}'),
            ),
          ),
        );
      },
    );
  }
}

## widgents/lego_agent_builder.dart

: Agent의 시각적 조립을 수행하는 위젯

In [ ]:
import 'package:flutter/material.dart';
import 'package:provider/provider.dart';
import '../models/crew_model.dart';

class LegoAgentBuilder extends StatelessWidget {
  final int agentIndex;

  LegoAgentBuilder({required this.agentIndex});

  @override
  Widget build(BuildContext context) {
    return Consumer<CrewModel>(
      builder: (context, crewModel, child) {
        final agent = crewModel.agents[agentIndex];
        return Column(
          children: [
            Image.asset(
              'assets/agent_${agent.name.toLowerCase()}.png',
              height: 200,
            ),
            Text(agent.name),
            Text(agent.role),
            SizedBox(height: 20),
            Text('Tasks:'),
            ...agent.tasks.map((task) => Text(task.name)),
            SizedBox(height: 20),
            Text('Tools:'),
            ...agent.tools.map((tool) => Image.asset(
                  'assets/tool_${tool.name.toLowerCase().replaceAll(' ', '_')}.png',
                  height: 50,
                )),
          ],
        );
      },
    );
  }
}


## widgets/parameter_sidebar.dart

: Agent의 세부 속성을 설정하는 위젯

: 우측 파라미터 사이드 바

: Task, Tool은 구현하는 메소드에 맞게 추가 해야함

## Agent 생성 화면에서 우측 사이드 바로 변경

In [ ]:
import 'package:flutter/material.dart';
import 'package:provider/provider.dart';
import '../models/crew_model.dart';

class ParameterSidebar extends StatelessWidget {
  final int agentIndex;

  ParameterSidebar({required this.agentIndex});

  @override
  Widget build(BuildContext context) {
    return Consumer<CrewModel>(
      builder: (context, crewModel, child) {
        final agent = crewModel.agents[agentIndex];
        return ListView(
          children: [
            _buildAgentSection(agent, crewModel),
            _buildTasksSection(agent, crewModel),
            _buildToolsSection(agent, crewModel),
          ],
        );
      },
    );
  }

  Widget _buildAgentSection(AgentModel agent, CrewModel crewModel) {
    return ExpansionTile(
      title: Text('Agent Parameters'),
      children: [
        TextField(
          decoration: InputDecoration(labelText: 'Agent Name'),
          controller: TextEditingController(text: agent.name),
          onChanged: (value) {
            crewModel.updateAgent(agent, name: value);
          },
        ),
        TextField(
          decoration: InputDecoration(labelText: 'Agent Role'),
          controller: TextEditingController(text: agent.role),
          onChanged: (value) {
            crewModel.updateAgent(agent, role: value);
          },
        ),
        TextField(
          decoration: InputDecoration(labelText: 'Agent Goal'),
          controller: TextEditingController(text: agent.goal),
          onChanged: (value) {
            crewModel.updateAgent(agent, goal: value);
          },
        ),
        TextField(
          decoration: InputDecoration(labelText: 'Agent Backstory'),
          controller: TextEditingController(text: agent.backstory),
          onChanged: (value) {
            crewModel.updateAgent(agent, backstory: value);
          },
        ),
      ],
    );
  }

  Widget _buildTasksSection(AgentModel agent, CrewModel crewModel) {
    return ExpansionTile(
      title: Text('Tasks'),
      children: [
        ...agent.tasks.asMap().entries.map((entry) {
          int idx = entry.key;
          TaskModel task = entry.value;
          return ExpansionTile(
            title: Text('Task ${idx + 1}'),
            children: [
              TextField(
                decoration: InputDecoration(labelText: 'Task Name'),
                controller: TextEditingController(text: task.name),
                onChanged: (value) {
                  crewModel.updateTask(agent, task, name: value);
                },
              ),
              TextField(
                decoration: InputDecoration(labelText: 'Task Description'),
                controller: TextEditingController(text: task.description),
                onChanged: (value) {
                  crewModel.updateTask(agent, task, description: value);
                },
              ),
              TextField(
                decoration: InputDecoration(labelText: 'Expected Output'),
                controller: TextEditingController(text: task.expectedOutput),
                onChanged: (value) {
                  crewModel.updateTask(agent, task, expectedOutput: value);
                },
              ),
            ],
          );
        }),
        ElevatedButton(
          child: Text('Add Task'),
          onPressed: () {
            crewModel.addTask(agent, TaskModel());
          },
        ),
      ],
    );
  }

  Widget _buildToolsSection(AgentModel agent, CrewModel crewModel) {
    return ExpansionTile(
      title: Text('Tools'),
      children: [
        ...agent.tools.asMap().entries.map((entry) {
          int idx = entry.key;
          ToolModel tool = entry.value;
          return ExpansionTile(
            title: Text('Tool ${idx + 1}'),
            children: [
              TextField(
                decoration: InputDecoration(labelText: 'Tool Name'),
                controller: TextEditingController(text: tool.name),
                onChanged: (value) {
                  crewModel.updateTool(agent, tool, name: value);
                },
              ),
              TextField(
                decoration: InputDecoration(labelText: 'Tool Description'),
                controller: TextEditingController(text: tool.description),
                onChanged: (value) {
                  crewModel.updateTool(agent, tool, description: value);
                },
              ),
            ],
          );
        }),
        ElevatedButton(
          child: Text('Add Tool'),
          onPressed: () {
            crewModel.addTool(agent, ToolModel());
          },
        ),
      ],
    );
  }
}

## models/crew_model.dart

: Crew, Agent, Task들의 상태를 관리하기 위한 모듈

**: LLM 등 디테일한 파라미터들 추가해야 함**

In [ ]:
import 'package:flutter/foundation.dart';

class CrewModel extends ChangeNotifier {
  String crewName = '';
  List<AgentModel> agents = [];

  void setCrewName(String name) {
    crewName = name;
    notifyListeners();
  }

  void addAgent(AgentModel agent) {
    agents.add(agent);
    notifyListeners();
  }

  void updateAgent(AgentModel agent,
      {String? name, String? role, String? goal, String? backstory}) {
    if (name != null) agent.name = name;
    if (role != null) agent.role = role;
    if (goal != null) agent.goal = goal;
    if (backstory != null) agent.backstory = backstory;
    notifyListeners();
  }

  void addTask(AgentModel agent, TaskModel task) {
    agent.tasks.add(task);
    notifyListeners();
  }

  void updateTask(AgentModel agent, TaskModel task,
      {String? name, String? description, String? expectedOutput}) {
    if (name != null) task.name = name;
    if (description != null) task.description = description;
    if (expectedOutput != null) task.expectedOutput = expectedOutput;
    notifyListeners();
  }

  void addTool(AgentModel agent, ToolModel tool) {
    agent.tools.add(tool);
    notifyListeners();
  }

  void updateTool(AgentModel agent, ToolModel tool,
      {String? name, String? description}) {
    if (name != null) tool.name = name;
    if (description != null) tool.description = description;
    notifyListeners();
  }

  Map<String, dynamic> toJson() {
    return {
      'crewName': crewName,
      'agents': agents.map((agent) => agent.toJson()).toList(),
    };
  }

  void addDummyData() {
    crewName = "Test Crew";
    agents = [
      AgentModel(
        name: "Researcher",
        role: "Information Gatherer",
        goal: "Collect and organize data",
        backstory: "Experienced in data analysis and research methodologies",
        tasks: [
          TaskModel(
            name: "Web Research",
            description: "Gather information from reliable web sources",
            expectedOutput: "Comprehensive report on findings",
          ),
        ],
        tools: [
          ToolModel(
            name: "Web Scraper",
            description: "Tool for extracting data from websites",
          ),
        ],
      ),
      AgentModel(
        name: "Analyst",
        role: "Data Interpreter",
        goal: "Analyze and interpret collected data",
        backstory: "Expert in statistical analysis and data visualization",
        tasks: [
          TaskModel(
            name: "Data Analysis",
            description: "Perform statistical analysis on gathered data",
            expectedOutput:
                "Analytical report with insights and visualizations",
          ),
        ],
        tools: [
          ToolModel(
            name: "Statistical Software",
            description: "Advanced tool for statistical analysis",
          ),
        ],
      ),
    ];
    notifyListeners();
  }
}

class AgentModel {
  String name = '';
  String role = '';
  String goal = '';
  String backstory = '';
  List<TaskModel> tasks = [];
  List<ToolModel> tools = [];

  AgentModel({
    this.name = '',
    this.role = '',
    this.goal = '',
    this.backstory = '',
    List<TaskModel>? tasks,
    List<ToolModel>? tools,
  })  : tasks = tasks ?? [],
        tools = tools ?? [];

  Map<String, dynamic> toJson() {
    return {
      'name': name,
      'role': role,
      'goal': goal,
      'backstory': backstory,
      'tasks': tasks.map((task) => task.toJson()).toList(),
      'tools': tools.map((tool) => tool.toJson()).toList(),
    };
  }
}

class TaskModel {
  String name = '';
  String description = '';
  String expectedOutput = '';

  TaskModel({
    this.name = '',
    this.description = '',
    this.expectedOutput = '',
  });

  Map<String, dynamic> toJson() {
    return {
      'name': name,
      'description': description,
      'expectedOutput': expectedOutput,
    };
  }
}

class ToolModel {
  String name = '';
  String description = '';

  ToolModel({
    this.name = '',
    this.description = '',
  });

  Map<String, dynamic> toJson() {
    return {
      'name': name,
      'description': description,
    };
  }
}

## service/api_service.dart

: 백엔드 서버와의 호출을 연결하기 위한 모듈

## 토요일 미팅 시 한솔님 백엔드 서버 URL로 연동

In [ ]:
import 'package:http/http.dart' as http;
import 'dart:convert';

class ApiService {
  final String baseUrl = 'https://your-backend-url.com/api';

  Future<Map<String, dynamic>> createWorkflow(Map<String, dynamic> data) async {
    final response = await http.post(
      Uri.parse('$baseUrl/create-workflow'),
      headers: {'Content-Type': 'application/json'},
      body: json.encode(data),
    );

    if (response.statusCode == 200) {
      return json.decode(response.body);
    } else {
      throw Exception('Failed to create workflow');
    }
  }
}